In [63]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import requests
import QuantLib as ql
import numpy as np
import matplotlib.pyplot as plt
import json
from sklearn.metrics import mean_squared_error
from scipy.optimize import differential_evolution
import concurrent.futures
from tradingview_screener import Query, col
import rookiepy
from sklearn.preprocessing import MinMaxScaler
from gbm_optimizer import optimize_gbm, gbm



with open("config.json", "r") as config_file:
    config = json.load(config_file)

api_key = config.get("api_key")
secret_key = config.get("secret_key")

pd.set_option('display.max_rows', None)

our_picks = ["AAPL", "AMD", "CHWY", "IBIT", "ASO", "GOOGL"]



In [64]:
NASDAQ = pd.read_csv('Indexes/NASDAQ.csv')
DOWJ = pd.read_csv('Indexes/DOWJ.csv')
SP = pd.read_csv('Indexes/S&P500.csv')

def clean_data(df):
    df = df[['Company', 'Symbol']]
    df = pd.DataFrame(df).dropna()
    return df

NASDAQ = clean_data(NASDAQ)
DOWJ = clean_data(DOWJ)
SP = clean_data(SP)


In [65]:
    
#TODO FIX SCREENER 
def screen_stocks():
    # Get cookies for TradingView session
    cookies = rookiepy.to_cookiejar(rookiepy.chrome(['.tradingview.com']))
    
    # Fetch stock data
    _, df = Query().select('close','change', 'Perf.3M').where(
        col('close').between(20, 55),
        col('change').between(-4,-2),
        col('Perf.3M') > 0,
        col('exchange').isin(['AMEX', 'CBOE', 'NASDAQ', 'NYSE']),

        ).limit(1000).get_scanner_data(cookies=cookies)
    
    df[['exchange', 'ticker']] = df['ticker'].str.split(':', expand=True)
    
    return df


def get_rolling_price_change_avg(ticker: str, days: int):
    try:
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days+10)
        
        data = yf.download(ticker, start=start_date, end=end_date, progress=False)
        
        if data.empty:
            return None, None
        
        data = data.sort_index()
        current_price = get_current_stock_price(ticker)

        data['Price_Change'] = ((current_price - data['Close'].shift(1)) / data['Close'].shift(1)) * 100

        rolling_avg = data['Price_Change'].rolling(window=min(days, len(data))).mean().iloc[-1]

        return rolling_avg
    
    except Exception as e:
        print(f"Error occurred for ticker {ticker}: {e}")
        return None, None

def get_current_stock_price(symbol: str):

    url = "https://data.alpaca.markets/v2/stocks/trades/latest"

    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": secret_key,
    }

    params = {
        "symbols": symbol,  
        "feed": "iex" 
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  

        data = response.json()
        return data.get("trades", {}).get(symbol, {}).get("p") 

    except requests.exceptions.RequestException as e:
        print(f"Error fetching stock price: {e}")


def get_option_chain(api_key: str, secret_key: str, ticker: str, expiration_date: datetime):
    expiration_str = expiration_date.strftime("%Y-%m-%d")  # Convert datetime to string
    
    url = f"https://data.alpaca.markets/v1beta1/options/snapshots/{ticker}?feed=indicative&limit=100&expiration_date={expiration_str}"
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": secret_key,
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        option_chain = data.get("snapshots", {})

        if not option_chain:
            return None

        parsed_data = []
        for symbol, details in option_chain.items():
            expiration_start = len(symbol) - 15
            option_type = "Call" if symbol[expiration_start+6] == "C" else "Put"
            strike_price = int(symbol[expiration_start+7:]) / 1000  

            greeks = details.get("greeks", {}) or {}
            latest_quote = details.get("latestQuote", {})

            parsed_data.append({
                "symbol": ticker,
                "expiration_date": expiration_str,  # Use the formatted string
                "option_type": option_type,
                "strike_price": strike_price,
                "delta": greeks.get("delta"),
                "gamma": greeks.get("gamma"),
                "rho": greeks.get("rho"),
                "theta": greeks.get("theta"),
                "vega": greeks.get("vega"),
                "implied_volatility": details.get("impliedVolatility"),
                "ask_price": latest_quote.get("ap"),
                "ask_size": latest_quote.get("as"),
                "bid_price": latest_quote.get("bp"),
                "bid_size": latest_quote.get("bs"),
            })

        return pd.DataFrame(parsed_data)

    except requests.exceptions.RequestException as e:
        print(f"Error fetching option chain: {e}")
        return None
    


def compute_score(contracts):
    """Compute the weighted score for contracts using normalized values."""
    temp_contracts = contracts.copy()
    
    scaler = MinMaxScaler()
    temp_contracts[['profitability_likelihood', 'return_percent', 'sharpe_ratio']] = scaler.fit_transform(
        temp_contracts[['profitability_likelihood', 'return_percent', 'sharpe_ratio']]
    )
    
    temp_contracts['score'] = (
        0.50 * temp_contracts['profitability_likelihood'] +
        0.25 * temp_contracts['return_percent'] +
        0.25 * temp_contracts['sharpe_ratio']
    )
    
    contracts['score'] = temp_contracts['score']
    
    return contracts

def select_optimal_contract(contracts):
    """Select the contract with the highest weighted score while keeping original values."""
    contracts = compute_score(contracts)
    contracts = contracts.sort_values(by='score', ascending=False)
    return contracts

def audit_contracts(expiration_date: datetime):
    # t-bill 3-month rate: 4.19%, inflation rate: 2.9% -> scaled to weekly
    risk_free_rate = (((1 + 0.0419) / (1 + 0.029)) ** (1/52) - 1) * 100
    simulation_attempts = 250
    optimizer_training_period = "1y"
    bin_length = 18

    all_options = pd.DataFrame(columns=[
        'symbol', 'expiration_date', 'option_type', 'strike_price', 'delta', 
        'gamma', 'rho', 'theta', 'vega', 'implied_volatility', 'ask_price', 
        'ask_size', 'bid_price', 'bid_size'
    ])
    
    canidates = screen_stocks()['ticker'].to_list()
    canidates.extend(our_picks)    
    print(canidates)

    for symbol in canidates:
        print(f"Evaluating {symbol}")
        option_chain = get_option_chain(api_key=api_key, secret_key=secret_key, ticker=symbol, expiration_date=expiration_date)

        if option_chain is None or option_chain.empty:
            continue 
        
        put_chain = option_chain[
            (option_chain['option_type'] == 'Put') & (option_chain['rho'].notna())
        ].sort_values(by='strike_price', ascending=True)

        price = get_current_stock_price(symbol)
        optimized_mu, optimized_sigma = optimize_gbm(symbol=symbol, training_period=optimizer_training_period, bin_length=bin_length)

        profitability_chances = []
        percent_returns = []

        for index, contract in put_chain.iterrows():
            count = 0
            strike_price = contract['strike_price']
            
            premium_collected = (contract['bid_price'] + contract['ask_price']) / 2   

            for _ in range(simulation_attempts):
                prices = gbm(
                    s0=price, mu=optimized_mu, sigma=optimized_sigma, 
                    deltaT=np.busday_count(datetime.today().date(), datetime.strptime(contract['expiration_date'], "%Y-%m-%d").date()), 
                    dt=1
                )

                if prices[-1] > strike_price:
                    count += 1

            profitability_chance = (count / simulation_attempts) * 100
            percent_return = (premium_collected / strike_price) * 100  

            put_chain.at[index, 'profitability_likelihood'] = profitability_chance
            put_chain.at[index, 'return_percent'] = percent_return
            put_chain.at[index, 'current_price'] = price
            put_chain.at[index, 'fill_price'] = premium_collected

            if put_chain['return_percent'].std() != 0:
                put_chain.at[index, 'sharpe_ratio'] = (percent_return - risk_free_rate) / put_chain['return_percent'].std()
            else:
                put_chain.at[index, 'sharpe_ratio'] = 0  

        all_options = pd.concat([all_options, put_chain], ignore_index=True, copy=False)

    return all_options


In [66]:

priced_contracts = audit_contracts(expiration_date = datetime(year=2025, month=2, day=14))

['BITX', 'ARCC', 'BITU', 'GME', 'MP', 'CONL', 'NTR', 'HUT', 'DXC', 'QFIN', 'PGNY', 'UCO', 'AESI', 'DXYZ', 'MEOH', 'TBT', 'TECX', 'ARQQ', 'TWM', 'EDN', 'MAGX', 'ALGS', 'BTCL', 'TGS', 'QQQU', 'MAXI', 'TESL', 'BTFX', 'YCS', 'SRS', 'DRAG', 'UCC', 'HDL', 'PFX', 'GBLI', 'MUD', 'EVAV', 'UX', 'CNFRZ', 'AAPL', 'AMD', 'CHWY', 'IBIT', 'ASO', 'GOOGL']
Evaluating AAPL


[*********************100%***********************]  1 of 1 completed
/var/folders/90/x6spqd2j4cbb38yrckqjm8840000gn/T/ipykernel_5442/2113592757.py:213: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_options = pd.concat([all_options, put_chain], ignore_index=True, copy=False)


In [67]:
contracts = select_optimal_contract(priced_contracts)
print('length',len(contracts))
display(contracts)

length 30


,symbol,expiration_date,option_type,strike_price,delta,gamma,rho,theta,vega,implied_volatility,ask_price,ask_size,bid_price,bid_size,profitability_likelihood,return_percent,current_price,fill_price,sharpe_ratio,score
15,AAPL,2025-02-14,Put,222.5,-0.1548,0.0234,-0.0081,-0.1473,0.0819,0.2964,0.85,86,0.83,19,99.2,0.377528,232.45,0.840,3.537861,0.742544
13,AAPL,2025-02-14,Put,217.5,-0.0809,0.0134,-0.0042,-0.1028,0.0515,0.3260,0.43,50,0.41,69,100.0,0.193103,232.45,0.420,3.450485,0.736244
14,AAPL,2025-02-14,Put,220.0,-0.1096,0.0177,-0.0057,-0.1209,0.0645,0.3076,0.60,45,0.54,21,100.0,0.259091,232.45,0.570,3.421904,0.735849
16,AAPL,2025-02-14,Put,225.0,-0.2106,0.0300,-0.0110,-0.1675,0.0992,0.2796,1.18,4,1.16,11,96.0,0.520000,232.45,1.170,3.519018,0.728613
12,AAPL,2025-02-14,Put,215.0,-0.0552,0.0097,-0.0029,-0.0787,0.0384,0.3348,0.28,51,0.27,80,100.0,0.127907,232.45,0.275,3.066126,0.708498
17,AAPL,2025-02-14,Put,227.5,-0.2892,0.0369,-0.0151,-0.1900,0.1175,0.2697,1.73,16,1.70,45,87.2,0.753846,232.45,1.715,3.610676,0.696368
10,AAPL,2025-02-14,Put,210.0,-0.0361,0.0060,-0.0019,-0.0649,0.0272,0.3872,0.20,113,0.19,54,100.0,0.092857,232.45,0.195,2.795083,0.689189
9,AAPL,2025-02-14,Put,207.5,-0.0295,0.0047,-0.0015,-0.0586,0.0231,0.4123,0.17,14,0.16,19,100.0,0.079518,232.45,0.165,2.753413,0.686033
7,AAPL,2025-02-14,Put,202.5,-0.0207,0.0031,-0.0011,-0.0492,0.0172,0.4644,0.13,13,0.12,34,100.0,0.061728,232.45,0.125,2.665713,0.679633
11,AAPL,2025-02-14,Put,212.5,-0.0402,0.0072,-0.0021,-0.0642,0.0297,0.3514,0.21,213,0.19,90,100.0,0.094118,232.45,0.200,2.570331,0.673891


In [68]:
# TODO
"""
- Figure out way to normalize stock price, whether that is min max of the range of the price(shoudl help optimizer
- Find better metric for optimizer
- Try binning, so like get past 10 years of AAPL, seperate into bins of 20 or n trading days, train optimzer on each one. Then the hyperparameters can be weighted to have more bias towards more recent bins
"""




'\n- Figure out way to normalize stock price, whether that is min max of the range of the price(shoudl help optimizer\n- Find better metric for optimizer\n- Try binning, so like get past 10 years of AAPL, seperate into bins of 20 or n trading days, train optimzer on each one. Then the hyperparameters can be weighted to have more bias towards more recent bins\n'

In [69]:
# Archive


# def gbm(s0, mu, sigma, deltaT, dt):
#     """
#     Models a stock price S(t) using the Wiener process W(t) as
#     `S(t) = S(0).exp{(mu-(sigma^2/2).t)+sigma.W(t)}`
    
#     Arguments:
#         s0: Initial stock price, default 100
#         mu: 'Drift' of the stock (upwards or downwards), default 0.2
#         sigma: 'Volatility' of the stock, default 0.68
#         deltaT: The time period for which the future prices are computed, default 52 (as in 52 weeks)
#         dt: The granularity of the time-period, default 0.1
    
#     Returns:
#         time_vector: array of time steps
#         s: array with the simulated stock prices over the time-period deltaT
#     """
#     n_step = int(deltaT / dt)  # Number of time steps
#     time_vector = np.linspace(0, deltaT, num=n_step)  # Time vector
    
#     # Wiener process: cumulative sum of random normal increments
#     random_increments = np.random.normal(0, 1, size=n_step) * np.sqrt(dt)
#     weiner_process = np.cumsum(random_increments)
    
#     # Stock price simulation
#     stock_var = (mu - (sigma**2 / 2)) * time_vector
#     s = s0 * np.exp(stock_var + sigma * weiner_process)
    
#     return s



# def aobjective(params, real_prices, s0):
#     """Objective function for optimization."""
#     mu, sigma = params  # Unpack parameters
#     gbm_prices = gbm(s0, mu, sigma, deltaT=len(real_prices), dt=1)
#     return mean_squared_error(real_prices, gbm_prices)

# def aoptimize_gbm(symbol: str, training_period: str, bin_length: int):
#     """
#     Optimize μ and σ over multiple time bins, weighting recent periods more.
#     """
#     # Fetch real stock data (past 5 years)
#     stock_data = yf.download(symbol, period=training_period, interval="1d")
#     real_prices = stock_data["Close"].dropna().values

#     num_bins = len(real_prices) // bin_length
#     weights = np.linspace(1, 2, num_bins)  # Increasing weights for recent bins

#     mu_values, sigma_values, mses = [], [], []

#     for i in range(num_bins):
#         bin_prices = real_prices[i * bin_length : (i + 1) * bin_length]
#         s0 = bin_prices[0]

#         bounds = [(-0.3, 0.3), (0.001, 0.35)]

#         result = differential_evolution(objective, bounds, args=(bin_prices, s0))
#         best_mu, best_sigma = result.x
#         best_mse = result.fun

#         mu_values.append(best_mu)
#         sigma_values.append(best_sigma)
#         mses.append(best_mse)

#     weight_sum = np.sum(weights)
#     avg_mu = np.sum(np.array(mu_values) * weights) / weight_sum
#     avg_sigma = np.sum(np.array(sigma_values) * weights) / weight_sum

#     print(f"\nFinal Weighted Averages: μ = {avg_mu:.4f}, σ = {avg_sigma:.4f}")

#     return avg_mu, avg_sigma



    # def filter_stocks(rolling_change_period): 
    # filtered_stocks = set()
    # stocks = screen_stocks()

    # for index, stock in stocks.iterrows():
    #     try:
    #         today_change, rolling_avg = get_rolling_price_change_avg(stock['ticker'], days=rolling_change_period)
    #         current_price = get_current_stock_price(stock['ticker'])

    #         # Skip if any value is None
    #         if None in (today_change, rolling_avg, current_price):
    #             print(f"Skipping {stock['ticker']} due to missing data.")
    #             continue

    #         # Apply filtering conditions
    #         if (rolling_avg > 0.00): 
    #             filtered_stocks.add(stock['ticker'])

    #     except Exception as e:
    #         print(f"Skipping {stock['ticker']} due to error: {e}")
    #         continue
    
    # return filtered_stocks

    #def ORIGINAL_LOGIC_FOR _AUDITING_OPTIONS()
    # simulation_attempts = 200
    # optimizer_training_period = "2y"
    # bin_length = 20
    # rolling_change_period = 15
    # expiration_date = datetime(year=2025, month=2, day=14) 
    # all_options = pd.DataFrame(columns=['symbol', 'expiration_date', 'option_type', 'strike_price', 'delta', 'gamma', 'rho', 'theta', 'vega', 'implied_volatility', 'ask_price', 'ask_size', 'bid_price', 'bid_size'])
    # candidates = ["AAPL", "AMD"]
    # # filter_stocks(rolling_change_period=rolling_change_period)

    # # t-bill 3-month rate: 4.19%, inflation rate: 2.9% -> scaled to weekly
    # risk_free_rate = (((1 + 0.0419) / (1 + 0.029)) ** (1/52) - 1) * 100

    # print(candidates)

    # for symbol in candidates:
    #     option_chain = get_option_chain(api_key=api_key, secret_key=secret_key, ticker=symbol, expiration_date=expiration_date)
    #     put_chain = option_chain[(option_chain['option_type'] == 'Put') & (option_chain['rho'].notna())].sort_values(by='strike_price', ascending=True)

    #     if option_chain is None or option_chain.empty:
    #         continue 

    #     price = get_current_stock_price(symbol)
    #     optimized_mu, optimized_sigma = optimize_gbm(symbol=symbol, training_period=optimizer_training_period, bin_length=bin_length)

    #     profitability_chances = []
    #     percent_returns = []

    #     for index, contract in put_chain.iterrows():
    #         count = 0
    #         strike_price = contract['strike_price']

    #         for i in range(simulation_attempts):
    #             prices = gbm(s0=price, mu=optimized_mu, sigma=optimized_sigma, 
    #                 deltaT=np.busday_count(datetime.today().date(), datetime.strptime(contract['expiration_date'], "%Y-%m-%d").date()), dt=1)  
    #             if prices[-1] > strike_price:
    #                 count += 1
    #         profitability_chance = (count / simulation_attempts) * 100
    #         profit = (contract['bid_price']*contract['bid_size'] + contract['ask_price']*contract['ask_size']) / (contract['ask_size'] + contract['bid_size'])
    #         percent_return = (profit / (strike_price)) * 100

    #         profitability_chances.append(profitability_chance)
    #         percent_returns.append(percent_return)
    #     put_chain['profitability_percent'] = profitability_chances
    #     put_chain['percent_return'] = percent_returns
    #     put_chain['expected_value'] = put_chain['profitability_percent'] * put_chain['percent_return']
    #     put_chain['current_price'] = price
    #     if put_chain['percent_return'].std() != 0:
    #         put_chain['sharpe_ratio'] = (put_chain['percent_return'] - risk_free_rate) / put_chain['percent_return'].std()
    #     else:
    #         put_chain['sharpe_ratio'] = 0  # Avoid division by zero
    #     all_options = pd.concat([all_options, put_chain], ignore_index=True, copy=False)


# def gbm_vs_real_graph(symbol, mu, sigma, period):
#     stock_data = yf.download(symbol, period=period, interval="1d")
#     real_prices = stock_data["Close"].dropna().values
#     time_steps = np.arange(len(real_prices))


#     gbm_path = gbm(s0 = real_prices[0], mu=mu, sigma=sigma, deltaT=len(real_prices), dt=1)
#     plt.figure(figsize=(10, 5))
#     plt.plot(time_steps, real_prices, label="Real Prices", color="blue")
#     plt.plot(time_steps, gbm_path, label="GBM Simulated", linestyle="dashed", color="red")
    
#     plt.xlabel("Time (Days)")
#     plt.ylabel("Price")
#     plt.title(f"GBM vs Real Prices for {symbol}")
#     plt.legend()
#     plt.grid()
#     plt.show()

# def multithread_optimize_bin(bin_prices, bin_size, weights, i):
#     s0 = bin_prices[0]

#     # Define the bounds for optimization
#     bounds = [(-0.3, 0.3), (0.001, 0.30)]

#     # Run the optimizer for the bin
#     result = differential_evolution(objective, bounds, args=(bin_prices, s0))
#     best_mu, best_sigma = result.x
#     best_mse = result.fun

#     print(f"Bin {i+1}: μ = {best_mu:.4f}, σ = {best_sigma:.4f}, MSE = {best_mse:.4f}")
#     return best_mu, best_sigma, best_mse

# def multithread_optimize_gbm(symbol): 
    # """
    # Optimize μ and σ over multiple time bins, weighting recent periods more.
    # """
    # # Fetch real stock data (past 2 years)
    # stock_data = yf.download(symbol, period="2y", interval="1d")
    # real_prices = stock_data["Close"].dropna().values

    # # Split into bins of 20 trading days
    # bin_size = 20
    # num_bins = len(real_prices) // bin_size
    # weights = np.linspace(1, 2, num_bins)  # Increasing weights for recent bins

    # # Initialize containers for results
    # mu_values, sigma_values, mses = [], [], []

    # # Use concurrent.futures for parallel processing of bins
    # with concurrent.futures.ThreadPoolExecutor() as executor:
    #     futures = []
    #     for i in range(num_bins):
    #         bin_prices = real_prices[i * bin_size : (i + 1) * bin_size]
    #         futures.append(executor.submit(optimize_bin, bin_prices, bin_size, weights, i))
        
    #     for future in concurrent.futures.as_completed(futures):
    #         best_mu, best_sigma, best_mse = future.result()
    #         mu_values.append(best_mu)
    #         sigma_values.append(best_sigma)
    #         mses.append(best_mse)

    # # Compute weighted averages
    # weight_sum = np.sum(weights)
    # avg_mu = np.sum(np.array(mu_values) * weights) / weight_sum
    # avg_sigma = np.sum(np.array(sigma_values) * weights) / weight_sum

    # print(f"\nFinal Weighted Averages: μ = {avg_mu:.4f}, σ = {avg_sigma:.4f}")

    # return avg_mu, avg_sigma